In [2]:
# Import required libraries
import pandas as pd
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [22]:

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("data/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                options=[
                                    {'label': 'All Sites', 'value': 'ALL'},
                                    {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                    {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                    {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                    {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                ],
                                value='ALL',
                                placeholder="Launch Site",
                                searchable=True
                                ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                html.Div(dcc.RangeSlider(id='payload-slider',
                                    min=0, max=10000, step=1000,
                                    marks={0: '0',
                                           2500: '2500',
                                           5000: "5000",
                                           7500: "7500",
                                           10000: "10000"},
                                    value=[min_payload, max_payload])),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                
                                html.Br(),
                                html.Div(dcc.Graph(id='booster-success-pie-chart')),
                                html.Br(),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
    # Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
                Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
        if entered_site == 'ALL':
            fig = px.pie(spacex_df, values='class', 
            names='Launch Site', 
            title='Total Success Launches By Site')
            # return the outcomes piechart for a selected site
            return fig
        else:
            data_selected = spacex_df[spacex_df["Launch Site"] == entered_site]
            d = {"Name": ["Success", "Failure"], "Success rate": [data_selected[data_selected["class"] == 1]["class"].count(), data_selected[data_selected["class"] == 0]["class"].count()]}
            data = pd.DataFrame(columns=["Name", "Success rate"], data = d)
            fig = px.pie(data, values='Success rate', 
            names="Name", 
            title='Total Launches for site ' + entered_site)
            # return the outcomes piechart for a selected site
            return fig
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
                [Input(component_id='site-dropdown', component_property='value'),
                 Input(component_id="payload-slider", component_property="value")],)
def get_success_rate(entered_site, payload):
        if entered_site == 'ALL':
            fig = px.scatter(spacex_df, x="Payload Mass (kg)" ,y="class",
            color="Booster Version Category",
            title='Correlation between Payload and Success for all sites')
            # return the outcomes piechart for a selected site
            fig.update_xaxes(range=payload)
            return fig
        else:
            data_selected = spacex_df[spacex_df["Launch Site"] == entered_site]
            fig = px.scatter(data_selected, x="Payload Mass (kg)" ,y="class",
            color="Booster Version Category",
            title='Correlation between Payload and Success for ' + entered_site)
            # return the outcomes piechart for a selected site
            fig.update_xaxes(range=payload)
            return fig

@app.callback(Output(component_id='booster-success-pie-chart', component_property='figure'),
                Input(component_id='site-dropdown', component_property='value'))
def get_booster_success(entered_site):
        if entered_site == 'ALL':
            fig = px.pie(spacex_df, values='class', 
            names='Booster Version Category', 
            title='Total Successful Launches By Booster Version')
            # return the outcomes piechart for a selected site
            return fig
        else:
            data_selected = spacex_df[spacex_df["Launch Site"] == entered_site]
            fig = px.pie(data_selected, values='class', 
            names='Booster Version Category', 
            title='Total Launches By Booster Version for Launch Site ' + entered_site)
            # return the outcomes piechart for a selected site
            return fig

# Run the app
if __name__ == '__main__':
    app.run(port=8055)
